<a href="https://colab.research.google.com/github/NicoleLund/flight_delay_prediction/blob/nrl_210817/data_manipulation_modeling/feature_assessment/feature_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

feature_assessment.ipynb
-----

Written in Google Colab

by Nicole Lund

This workbook investigates the correlation between features in 2017 flight performance prior to building a model from the data.

In [40]:
# Import dependencies
import pandas as pd
import datetime
from datetime import timedelta

In [41]:
# Read the CSV file from AWS to Pandas Dataframe
url = "https://finalproject-3.s3.us-west-1.amazonaws.com/2017_TUS.csv"
df = pd.read_csv(url)

df.head(3)

,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week,DELAY
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-01,DL,2066,TUS,ATL,1430,1459.0,29.0,10.0,1509.0,2011.0,17.0,2003,2028.0,25.0,0.0,NaN,0.0,213.0,209.0,182.0,1541.0,11.0,0.0,0.0,0.0,14.0,14:30,14:59,15:09,20:11,20:03,20:28,Sunday,0
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,1127,TUS,ATL,600,637.0,37.0,10.0,647.0,1157.0,6.0,1129,1203.0,34.0,0.0,NaN,0.0,209.0,206.0,190.0,1541.0,34.0,0.0,0.0,0.0,0.0,06:00,06:37,06:47,11:57,11:29,12:03,Monday,1
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,2066,TUS,ATL,1430,1447.0,17.0,10.0,1457.0,1943.0,9.0,2005,1952.0,-13.0,0.0,NaN,0.0,215.0,185.0,166.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:47,14:57,19:43,20:05,19:52,Monday,0


In [42]:
df.dest_city_name.unique()

array(['ATLANTA, GEORGIA, USA              ',
       'DENVER, COLORADO, USA              ',
       'DALLAS/FT.WORTH, TEXAS, USA        ',
       'HOUSTON, TEXAS, USA                ',
       'NEW YORK, NEW YORK, USA            ',
       'LAS VEGAS, NEVADA, USA             ',
       'LOS ANGELES, CALIFORNIA, USA       ',
       'CHICAGO, ILLINOIS, USA             ',
       'MINNEAPOLIS/ST.PAUL, MINNESOTA, USA',
       'OAKLAND, CALIFORNIA, USA           ',
       'PORTLAND, OREGON, USA              ',
       'PHOENIX, ARIZONA, USA              ',
       'SAN DIEGO, CALIFORNIA, USA         ',
       'SEATTLE, WASHINGTON, USA           ',
       'SAN FRANCISCO, CALIFORNIA, USA     ',
       'SAN JOSE, CALIFORNIA, USA          ',
       'SALT LAKE CITY, UTAH, USA          '], dtype=object)

In [43]:
# List all of the column headers
df.columns

Index(['origin_city_name', 'dest_city_name', 'FL_DATE', 'OP_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'CRS_DEP_HM', 'DEP_TIME_HM', 'WHEELS_OFF_HM', 'WHEELS_ON_HM',
       'CRS_ARR_TIME_HM', 'ARR_TIME_HM', 'day_of_week', 'DELAY'],
      dtype='object')

In [44]:
# Identify row to verify calculations with
row = 0

In [45]:
# Verify DEP_DELAY/DEP_TIME calculation

print('Scheduled Departure Time')
print(df.CRS_DEP_TIME[row])
print(pd.to_datetime(df.CRS_DEP_TIME[row], format = '%H%M'))
print('')
print('Actual Departure Time')
print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M'))
# print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M').time())
# print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M').strftime(format = '%H:%M:%S'))
print('')

dep_delay = pd.to_datetime(df.DEP_TIME[row], format = '%H%M') - pd.to_datetime(df.CRS_DEP_TIME[row], format = '%H%M')
print('Departure Delay')
print(dep_delay)
print(df.DEP_DELAY[row])

Scheduled Departure Time
1430
1900-01-01 14:30:00

Actual Departure Time
1900-01-01 14:59:00

Departure Delay
0 days 00:29:00
29.0


In [46]:
# Verify WHEELS_OFF time
print('Departure time: ')
print(df.DEP_TIME[row])
print('Taxi time: ')
print(df.TAXI_OUT[row])
print('WHEELS OFF time:')
print(df.WHEELS_OFF[row])

Departure time: 
1459.0
Taxi time: 
10.0
WHEELS OFF time:
1509.0


In [47]:
# Verify WHEELS_ON time
print('Arrival time: ')
print(df.ARR_TIME[row])
print('Taxi time: ')
print(df.TAXI_IN[row])
print('WHEELS ON time:')
print(df.WHEELS_ON[row])

Arrival time: 
2028.0
Taxi time: 
17.0
WHEELS ON time:
2011.0


In [48]:
# Verify WHEELS_ON time
print('WHEELS OFF time:')
print(df.WHEELS_OFF[row])
print('AIR_TIME: ')
print(df.AIR_TIME[row])
print('hours')
print(182/60)
print('minutes')
print(182-3*60)
print('WHEELS_ON time: ')
print(df.WHEELS_ON[row])

WHEELS OFF time:
1509.0
AIR_TIME: 
182.0
hours
3.033333333333333
minutes
2
WHEELS_ON time: 
2011.0


In [49]:
# Verify ACTUAL_ELAPSED_TIME calculation
actual_elapsed_time = df.TAXI_OUT[row]+df.TAXI_IN[row]+df.AIR_TIME[row]
print(actual_elapsed_time)
print(df.ACTUAL_ELAPSED_TIME[row])

209.0
209.0


In [50]:
# Verify Arrival time
print('Departure time: ')
print(df.DEP_TIME[row])
print('Actual Elapsed Time: ')
print(df.ACTUAL_ELAPSED_TIME[row])
print('hours')
print(209/60)
print('minutes')
print(209-3*60)
print('Arrival time: ')
print(df.ARR_TIME[row])
print(df.origin_city_name[row])
print(df.dest_city_name[row])

Departure time: 
1459.0
Actual Elapsed Time: 
209.0
hours
3.4833333333333334
minutes
29
Arrival time: 
2028.0
TUCSON, ARIZONA, USA               
ATLANTA, GEORGIA, USA              


In [51]:
# Verify ARR_DELAY calculation
arr_delay = df.CARRIER_DELAY[row]+df.WEATHER_DELAY[row] + df.NAS_DELAY[row] + df.SECURITY_DELAY[row] + df.LATE_AIRCRAFT_DELAY[row]
print(arr_delay)
print(df.ARR_DELAY[row])

25.0
25.0


In [54]:
# Verify if ARR_DELAY is directly calculable from X values
print('Departure Time')
print(df.CRS_DEP_TIME[row])
print('Transit time')
print(df.DEP_DELAY[row] + df.TAXI_OUT[row] + df.AIR_TIME[row] + df.TAXI_IN[row])
print('hours')
print(238/60)
print('minutes')
print(238-3*60)
print(df.origin_city_name[row])
print(df.dest_city_name[row])
print('2hr time zone difference')
print('Calculated arrival time:')
print(14+3+2+1)
print(30-2)
print('Arrival time: ')
print(df.ARR_TIME[row])

Departure Time
1430
Transit time
238.0
hours
3.966666666666667
minutes
58
TUCSON, ARIZONA, USA               
ATLANTA, GEORGIA, USA              
2hr time zone difference
Calculated arrival time:
20
28
Arrival time: 
2028.0


In [55]:
# Review Diverted Values
print(df.DIVERTED.unique())
print(df.ARR_DELAY[df.DIVERTED == 1][:3])

[0. 1.]
51    NaN
329   NaN
362   NaN
Name: ARR_DELAY, dtype: float64


In [56]:
# Review CANCELLED Values
print(df.CANCELLED.unique())
print(df.ARR_DELAY[df.CANCELLED == 1][:3])

[0. 1.]
41    NaN
172   NaN
176   NaN
Name: ARR_DELAY, dtype: float64


In [57]:
df.loc[41,:]

origin_city_name       TUCSON, ARIZONA, USA               
dest_city_name         ATLANTA, GEORGIA, USA              
FL_DATE                                         2017-01-23
OP_CARRIER                                              DL
OP_CARRIER_FL_NUM                                     2636
ORIGIN                                                 TUS
DEST                                                   ATL
CRS_DEP_TIME                                           700
DEP_TIME                                               NaN
DEP_DELAY                                              NaN
TAXI_OUT                                               NaN
WHEELS_OFF                                             NaN
WHEELS_ON                                              NaN
TAXI_IN                                                NaN
CRS_ARR_TIME                                          1230
ARR_TIME                                               NaN
ARR_DELAY                                              N

In [58]:
# Review CANCELLATION_CODE Values
codes = df.CANCELLATION_CODE.unique()
for code in codes:
  num_found = df.ARR_DELAY[df.CANCELLATION_CODE == code].count()
  print(f"Code {code}: {num_found}")

Code nan: 0
Code B: 0
Code A: 0
Code C: 0


## Logical assessment of features

**Features to include in the model**

X values
* 'OP_CARRIER': airline designation
* 'OP_CARRIER_FL_NUM': flight number
* 'day_of_week': flight day of the week
* 'DEST': destination airport code
* 'CRS_DEP_TIME': scheduled departure time 
* 'CRS_ARR_TIME': scheduled arrival time
* 'DISTANCE': flight distance

Y values
* 'CANCELLED': flight cancelled, [0, 1]
* 'DIVERTED': flight diverted, [0, 1]
* 'DELAYED': arrival time delay
  * 0 = Delayed <30 minutes
  * 1 = Delayed >=30 minutes

**Features not to include in the model due to irrelevance**
* 'Unnamed: 0': extra index column
* 'ORIGIN': departure city, filtered for TUS only
* 'origin_city': departure city calculated from ORIGIN
* 'dest_city': destination city calculated from DEST
* 'CRS_ELAPSED_TIME': scheduled elapsed time
* 'WHEELS_OFF': actual wheels off time, DEP_TIME + TAXI_OUT
* 'WHEELS_ON': actual wheels on time, ARR_TIME - TAXI_IN
* 'CANCELLATION_CODE': reason for cancellation
* 'CARRIER_DELAY': arrival delay time due to carrier
* 'WEATHER_DELAY': arrival delay time due to weather
* 'NAS_DELAY': arrival delay time due to NAS
* 'SECURITY_DELAY': arrival delay time due to security
* 'LATE_AIRCRAFT_DELAY': arrival delay time due to aircraft

**Features not to include in the model due to calculations**
* 'FL_DATE': flight date provides day_of_week
* 'DEP_TIME': actual departure time, CRS_DEP_TIME + DEP_DELAY
* 'ACTUAL_ELAPSED_TIME': TAXI_OUT + TAXI_IN + AIR_TIME
* 'ARR_TIME': actual arrival time, DEP_TIME + ACTUAL_ELAPSED_TIME + time zone difference

**Features not to include in the model due to they directly reveal Y values**
* 'DEP_DELAY': departure delay time
* 'TAXI_OUT': calculated time spent in taxi between departure time and wheels off
* 'TAXI_IN': calculated time spent in taxi between wheels on and arrival time
* 'AIR_TIME': calculated time spent in the air



In [59]:
correlation_df = df.corr()
correlation_df

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DELAY
OP_CARRIER_FL_NUM,1.000000,-0.066724,-0.063815,0.028879,0.227720,-0.054537,-0.181288,0.000961,-0.195604,-0.181309,0.043600,0.020024,-0.004954,-0.192408,-0.175889,-0.218009,-0.285792,-0.009331,0.011555,0.107222,0.030998,-0.019211,0.067393
CRS_DEP_TIME,-0.066724,1.000000,0.983887,0.083546,-0.028142,0.983081,0.896652,-0.064532,0.912463,0.895679,0.072745,0.021577,0.005037,-0.168831,-0.176306,-0.165163,-0.163380,-0.121998,-0.026691,-0.143516,-0.032537,0.273526,0.096566
DEP_TIME,-0.063815,0.983887,1.000000,0.190488,-0.019420,0.997654,0.907676,-0.065094,0.899384,0.906107,0.176132,0.023013,0.005945,-0.160073,-0.166617,-0.156618,-0.156207,-0.021424,-0.002602,-0.117054,-0.034985,0.387983,0.191967
DEP_DELAY,0.028879,0.083546,0.190488,1.000000,0.069990,0.188204,0.152639,-0.015519,0.082011,0.149246,0.968493,0.019800,0.001635,0.016477,0.021119,0.012409,0.006400,0.728821,0.053509,0.339505,-0.014802,0.380538,0.600451
TAXI_OUT,0.227720,-0.028142,-0.019420,0.069990,1.000000,0.008095,-0.006435,0.088333,-0.038606,-0.003650,0.190789,-0.008300,0.010375,0.039868,0.167362,-0.009501,-0.043829,-0.003829,0.019692,0.197085,-0.022425,-0.097796,0.195688
WHEELS_OFF,-0.054537,0.983081,0.997654,0.188204,0.008095,1.000000,0.907548,-0.063042,0.896901,0.906154,0.177925,-0.000145,0.006002,-0.162188,-0.165296,-0.160179,-0.160727,-0.035637,-0.002099,-0.106381,-0.035256,0.389522,0.195346
WHEELS_ON,-0.181288,0.896652,0.907676,0.152639,-0.006435,0.907548,1.000000,-0.051835,0.974900,0.998010,0.139515,NaN,0.029574,0.165557,0.156495,0.169596,0.185395,-0.042236,-0.004945,-0.127528,-0.038890,0.347328,0.169975
TAXI_IN,0.000961,-0.064532,-0.065094,-0.015519,0.088333,-0.063042,-0.051835,1.000000,-0.045573,-0.027835,0.098514,NaN,0.009928,0.023261,0.141320,-0.029626,-0.024160,-0.053285,0.051053,0.152310,0.010234,-0.103091,0.086042
CRS_ARR_TIME,-0.195604,0.912463,0.899384,0.082011,-0.038606,0.896901,0.974900,-0.045573,1.000000,0.974307,0.059482,0.027174,0.016789,0.179207,0.158118,0.175550,0.199614,-0.069141,-0.033811,-0.130604,-0.034776,0.272065,0.078651
ARR_TIME,-0.181309,0.895679,0.906107,0.149246,-0.003650,0.906154,0.998010,-0.027835,0.974307,1.000000,0.138790,NaN,0.029368,0.164826,0.158415,0.167265,0.183407,-0.044955,-0.000602,-0.138600,-0.037210,0.344294,0.170898
